<a href="https://colab.research.google.com/github/junghyun9108/2025_DL/blob/main/20250421_CNN%EA%B3%BC%EC%A0%9C_%EC%A0%9C%EC%B6%9C_%EC%9D%B4%EC%A4%91%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from google.colab import drive
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# 기본 데이터 전처리. 추가적인 전처리는 따로 정의하여 사용해주세요.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10 데이터셋 다운로드
dataset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/20240513/', train=True,
                                        download=True, transform=transform)

# 학습 데이터셋과 검증 데이터셋으로 분할
trainset, valset = random_split(dataset, [40000, 10000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,
                                        shuffle=False, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/20240513/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=4)

def calculate_accuracy(testloader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %.2f %%' % (100 * correct / total))

CIFAR10 분류 문제의 성능을 더욱 향상시키는 방법은 여러 가지가 있습니다:

데이터 증강(Data Augmentation): 데이터 증강은 기존의 학습 데이터를 변형하여 새로운 학습 데이터를 생성하는 방법입니다. 이를 통해 모델이 다양한 변형에 대해 더욱 견고해질 수 있습니다. PyTorch의 torchvision.transforms 모듈을 사용하면 간단하게 데이터 증강을 적용할 수 있습니다.

다른 모델 아키텍처 사용: 실습에 사용한 코드 외에도 다양한 CNN 아키텍처가 있습니다.ResNet, VGG, GoogLeNet, DenseNet 등 다른 모델 아키텍처를 사용할 수 있습니다.

하이퍼파라미터 튜닝: 학습률, 배치 크기, 에폭 수 등의 하이퍼파라미터를 조정하여 성능을 향상시킬 수 있습니다.

Early Stopping: 검증 세트의 성능이 더 이상 향상되지 않을 때 학습을 중단하는 기법입니다. 이를 통해 과적합을 방지할 수 있습니다.

Regularization: L1, L2 정규화나 Dropout과 같은 정규화 기법을 사용하여 과적합을 방지할 수 있습니다.

위 방법들을 사용하여,

1) 실습 코드 마지막에 기재된 Accuracy of the network on the test images: 75.83 %보다 높은 성능을 갖는 코드를 작성해주세요.

2) 그리고 본인이 작성한 코드가 왜 기존의 코드보다 좋은 성능이 나왔는지 서술해주세요.

In [12]:
import torchvision.transforms as transforms

# 데이터 증강을 위한 변환 정의
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # RandomCrop 추가
    transforms.RandomHorizontalFlip(),  # 랜덤하게 수평으로 뒤집기
    transforms.RandomRotation(15),  # 랜덤하게 회전 (최대 15도)
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 테스트 데이터에는 데이터 증강을 적용하지 않음
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR10 데이터셋 다운로드 및 로더 생성
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/20240513/', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/20240513/', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=4)

In [13]:
import torchvision.models as models

# ResNet18 모델 로드 및 수정
model = models.resnet18(pretrained=False).to(device)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10).to(device)  # CIFAR10에 맞게 출력층 수정

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)  # weight decay 추가
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)  # learning rate scheduler 추가

# Early stopping parameters
patience = 10
best_loss = None
early_stop = False
early_stop_count = 0

# 모델 훈련
epochs = 100
for epoch in range(epochs):  # 데이터셋을 여러번 반복
    if early_stop:
        break

    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # 200 미니배치마다 출력
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(valloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= i+1
    print(f'Validation loss: {val_loss:.3f}')

    scheduler.step()  # learning rate scheduler 적용

    # early stopping 조건 확인
    if best_loss is None or val_loss < best_loss:
        best_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1
        if early_stop_count >= patience:
            early_stop = True

print('Finished Training')

[1,   200] loss: 1.867
[1,   400] loss: 1.620
[1,   600] loss: 1.525
Validation loss: 1.341
[2,   200] loss: 1.377
[2,   400] loss: 1.329
[2,   600] loss: 1.301
Validation loss: 1.101
[3,   200] loss: 1.216
[3,   400] loss: 1.164
[3,   600] loss: 1.139
Validation loss: 0.957
[4,   200] loss: 1.100
[4,   400] loss: 1.059
[4,   600] loss: 1.062
Validation loss: 0.899
[5,   200] loss: 1.024
[5,   400] loss: 0.997
[5,   600] loss: 0.979
Validation loss: 0.875
[6,   200] loss: 0.956
[6,   400] loss: 0.939
[6,   600] loss: 0.927
Validation loss: 0.770
[7,   200] loss: 0.893
[7,   400] loss: 0.887
[7,   600] loss: 0.889
Validation loss: 0.725
[8,   200] loss: 0.871
[8,   400] loss: 0.846
[8,   600] loss: 0.861
Validation loss: 0.678
[9,   200] loss: 0.835
[9,   400] loss: 0.831
[9,   600] loss: 0.816
Validation loss: 0.659
[10,   200] loss: 0.797
[10,   400] loss: 0.806
[10,   600] loss: 0.790
Validation loss: 0.631
[11,   200] loss: 0.759
[11,   400] loss: 0.782
[11,   600] loss: 0.774
Valid

In [15]:
calculate_accuracy(testloader, model)

Accuracy of the network on the test images: 86.30 %


# 성능 향상 이유

1) 데이터 증강: 데이터 증강을 통해 학습 데이터의 다양성을 높여 모델이 과적합되는 것을 방지하고, 다양한 변형에 대해 더욱 잘 학습되도록 구성하였습니다.

2) ResNet18 모델: ResNet18은 CIFAR10 데이터셋에서 좋은 성능을 보이는 것으로 알려진 모델 아키텍처입니다. 잔차 연결(residual connection)을 사용하여 네트워크 학습 효율을 올렸습니다.

3) 하이퍼파라미터 튜닝: weight decay, learning rate scheduler 등을 적용하여 모델의 학습 과정을 최적화하고, 더 나은 성능을 뽑을 수 있도록 수정하였습니다.

4) Early Stopping: Early stopping을 통해 과적합을 방지하고, 검증 데이터셋에서 가장 좋은 성능을 보이는 모델을 선택하여 테스트 데이터셋에서도 좋은 성능을 얻을 수 있도록 작성하였습니다.